# Spatial Marginal Effects: The Most Important Part of Spatial Econometrics

**Duration**: 150-180 minutes  
**Level**: Advanced  
**Prerequisites**: Notebooks 01-05 (especially SAR and SDM)

---

## Learning Objectives

By the end of this notebook, you will:

1. **Understand** why coefficients β ≠ marginal effects in spatial models
2. **Compute** direct, indirect, and total spatial effects
3. **Interpret** the spatial multiplier S(ρ) = (I - ρW)⁻¹
4. **Distinguish** between SAR and SDM effects
5. **Apply** simulation-based vs delta method inference
6. **Evaluate** policy impacts using spatial effects

---

## Why This Matters

**The #1 mistake in applied spatial econometrics**: Interpreting β coefficients as marginal effects.

In spatial models, a change in x_i affects:
- y_i directly (own effect)
- y_j indirectly (spillover to neighbors)
- y_i again through feedback (neighbors' responses)

This creates a **spatial multiplier** that amplifies the direct effect. Ignoring spillovers can underestimate policy impacts by 50% or more!

---

In [ ]:
# Setup
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

# Add PanelBox to path
panelbox_path = Path("/home/guhaase/projetos/panelbox")
sys.path.insert(0, str(panelbox_path))

print("✓ Setup complete")

---

## Section 1: Why Coefficients ≠ Marginal Effects (25 minutes)

### The Fundamental Problem in Spatial Models

#### In Standard OLS

$$
y_i = \beta_0 + \beta_1 x_i + \varepsilon_i
$$

Marginal effect: $\frac{\partial y_i}{\partial x_i} = \beta_1$ ✓ Simple!

#### In Spatial Autoregressive (SAR) Model

$$
y = \rho W y + X\beta + \varepsilon
$$

Reduced form:
$$
y = (I - \rho W)^{-1} X\beta + (I - \rho W)^{-1} \varepsilon
$$

Marginal effect:
$$
\frac{\partial y}{\partial x_k} = (I - \rho W)^{-1} \beta_k = S(\rho) \beta_k
$$

**This is an N×N matrix, not a scalar!**

---

### Why the Difference?

**Feedback loops**: 

1. Change in x_i affects y_i (direct effect β)
2. y_i affects y_j via ρWy (spillover)
3. y_j feeds back to y_i via ρWy (feedback)
4. Infinite rounds converge to equilibrium

**Spatial multiplier**: S(ρ) = (I - ρW)⁻¹ = I + ρW + ρ²W² + ρ³W³ + ...

---

### Intuitive Example

**Scenario**: Region A increases education spending (x_A ↑ by 1%)

**Effects cascade**:
- Round 0: GDP growth in A increases by β (direct effect)
- Round 1: Growth in neighbor B increases by ρw_{BA}β (spillover)
- Round 2: Growth in A increases by ρw_{AB}ρw_{BA}β (feedback from B)
- Round 3: Growth in B increases by ρw_{BA}ρw_{AB}ρw_{BA}β (feedback continues)
- ...
- Equilibrium: Total effect = β × (1 + ρw + ρ²w² + ...) = β × spatial multiplier

---

In [ ]:
# Demonstrate feedback loop with simple example
print("=" * 70)
print("FEEDBACK LOOP DEMONSTRATION")
print("=" * 70)

# Two regions: A and B (neighbors)
# W matrix: each region has 50% weight on neighbor
W_2regions = np.array([
    [0.0, 0.5],  # A: 50% weight on B
    [0.5, 0.0]   # B: 50% weight on A
])

rho = 0.6  # Spatial autocorrelation
beta = 2.0  # Direct coefficient

# Shock: x_A increases by 1 unit
shock = np.array([1.0, 0.0])

# Simulate feedback rounds
y = np.zeros(2)
direct_effect = beta * shock  # Round 0
y += direct_effect

print(f"\nParameters: ρ = {rho}, β = {beta}")
print(f"Shock: x_A increases by 1 unit\n")
print(f"{'Round':<8} {'y_A':<12} {'y_B':<12} {'Total':<12} {'Description'}")
print("-" * 70)
print(f"{'0':<8} {y[0]:<12.4f} {y[1]:<12.4f} {y.sum():<12.4f} Direct effect (β)")

# Iterate feedback
for round_num in range(1, 11):
    spillover = rho * W_2regions @ y
    y += spillover
    
    if round_num <= 3 or round_num == 10:
        print(f"{round_num:<8} {y[0]:<12.4f} {y[1]:<12.4f} {y.sum():<12.4f} Feedback round {round_num}")
    elif round_num == 4:
        print(f"{'...':<8} {'...':<12} {'...':<12} {'...':<12} ...")

# Compare with analytical solution
I = np.eye(2)
S_rho = np.linalg.inv(I - rho * W_2regions)
y_analytical = S_rho @ (beta * shock)

print("-" * 70)
print(f"{'Analytical':<8} {y_analytical[0]:<12.4f} {y_analytical[1]:<12.4f} {y_analytical.sum():<12.4f} (I-ρW)⁻¹ β")
print("=" * 70)

print(f"\n📊 Interpretation:")
print(f"  → Region A's 1-unit increase in x creates:")
print(f"    - {y_analytical[0]:.3f} total impact on own growth (direct + feedback)")
print(f"    - {y_analytical[1]:.3f} total spillover to region B")
print(f"    - {y_analytical.sum():.3f} system-wide impact")
print(f"  → Spatial multiplier = {y_analytical.sum() / beta:.3f}x")
print(f"  → Ignoring spillovers underestimates by {100 * y_analytical[1] / y_analytical.sum():.1f}%")
print("=" * 70)

---

## Section 2: The Impact Matrix (30 minutes)

### Decomposing Spatial Effects

The **impact matrix** $\frac{\partial y}{\partial x_k}$ shows how changes in variable k in each region affect outcomes in all regions.

#### For SAR Model

$$
\frac{\partial y}{\partial x_k} = (I - \rho W)^{-1} \beta_k = S(\rho) \beta_k
$$

#### For SDM Model

$$
y = \rho W y + X\beta + WX\theta + \varepsilon
$$

$$
\frac{\partial y}{\partial x_k} = (I - \rho W)^{-1} [\beta_k I + \theta_k W] = S(\rho)[\beta_k I + \theta_k W]
$$

---

### Interpretation of Impact Matrix Elements

For N×N matrix $M = S(\rho)\beta_k$:

- **M_ij**: Impact of 1-unit change in x_k in region j on y in region i
- **Diagonal M_ii**: Own effect (direct + feedback loops)
- **Off-diagonal M_ij (i≠j)**: Cross effect (spillover from j to i)

---

In [ ]:
# Numerical example with 3 regions
print("=" * 70)
print("IMPACT MATRIX EXAMPLE (3 Regions)")
print("=" * 70)

# 3-region W matrix (row-normalized)
W_simple = np.array([
    [0.0, 0.5, 0.5],  # Region 0: neighbors with 1 and 2
    [0.5, 0.0, 0.5],  # Region 1: neighbors with 0 and 2
    [0.5, 0.5, 0.0]   # Region 2: neighbors with 0 and 1
])

rho = 0.4
beta_k = 2.0

# Compute spatial multiplier
I = np.eye(3)
S_rho = np.linalg.inv(I - rho * W_simple)

# Impact matrix
impact = S_rho * beta_k

print(f"\nParameters: ρ = {rho}, β_k = {beta_k}")
print(f"\nSpatial Multiplier S(ρ) = (I - ρW)⁻¹:")
print(S_rho)

print(f"\nImpact Matrix = S(ρ) × β_k:")
print(impact)

print("\n" + "=" * 70)
print("INTERPRETATION")
print("=" * 70)
print(f"\n📍 Diagonal elements (own effects):")
for i in range(3):
    print(f"   Region {i}: 1-unit ↑ in x_k → y_{i} ↑ by {impact[i,i]:.4f}")

print(f"\n🔗 Off-diagonal elements (spillovers):")
print(f"   1-unit ↑ in x_k in region 0:")
print(f"     → y_1 ↑ by {impact[1,0]:.4f} (spillover to region 1)")
print(f"     → y_2 ↑ by {impact[2,0]:.4f} (spillover to region 2)")

print(f"\n📊 Row sums (total impact on each region):")
for i in range(3):
    row_sum = impact[i, :].sum()
    print(f"   Region {i}: {row_sum:.4f} (1-unit ↑ in x_k everywhere → y_{i} ↑ by {row_sum:.4f})")

print(f"\n🌐 Column sums (total impact from each region):")
for j in range(3):
    col_sum = impact[:, j].sum()
    print(f"   Region {j}: {col_sum:.4f} (1-unit ↑ in x_k in region {j} → system y ↑ by {col_sum:.4f})")

print("=" * 70)

---

## Section 3: Three Types of Effects (30 minutes)

### Definitions (LeSage & Pace 2009)

The N×N impact matrix is too complex to interpret directly. We summarize it using three scalar measures:

#### 1. Direct Effect

Average of diagonal elements:
$$
\text{Direct} = \frac{1}{N} \sum_{i=1}^N [S(\rho)\beta_k]_{ii}
$$

**Interpretation**: Average impact on own region (includes feedback)

#### 2. Indirect Effect

Average of row sums excluding diagonal:
$$
\text{Indirect} = \frac{1}{N} \sum_{i=1}^N \sum_{j \neq i} [S(\rho)\beta_k]_{ij}
$$

**Interpretation**: Average spillover to all other regions

#### 3. Total Effect

$$
\text{Total} = \text{Direct} + \text{Indirect} = \frac{1}{N} \sum_{i=1}^N \sum_{j=1}^N [S(\rho)\beta_k]_{ij}
$$

**Interpretation**: System-wide impact

---

### Why These Definitions?

- **Direct ≠ β**: Diagonal includes feedback loops, not just initial direct effect
- **Indirect**: Captures externalities ignored in non-spatial models
- **Total**: Relevant for policy evaluation (total benefit vs cost)

---

In [ ]:
# Manual computation of effects
def compute_effects_manual(S, beta_k):
    """
    Compute direct, indirect, total effects manually.
    
    Parameters
    ----------
    S : ndarray
        Spatial multiplier matrix (I - ρW)⁻¹
    beta_k : float
        Coefficient for variable k
    
    Returns
    -------
    dict
        Direct, indirect, total effects
    """
    N = S.shape[0]
    impact_matrix = S * beta_k
    
    # Direct effect: average diagonal
    direct = np.trace(impact_matrix) / N
    
    # Total effect: average of all elements
    total = impact_matrix.sum() / N
    
    # Indirect effect: total - direct
    indirect = total - direct
    
    return {
        'direct': direct,
        'indirect': indirect,
        'total': total,
        'impact_matrix': impact_matrix
    }


# Apply to previous example
effects = compute_effects_manual(S_rho, beta_k)

print("=" * 70)
print("EFFECTS DECOMPOSITION")
print("=" * 70)
print(f"\nParameters: N = 3 regions, ρ = {rho}, β_k = {beta_k}")
print(f"\n{'Effect':<20} {'Value':<12} {'Formula'}")
print("-" * 70)
print(f"{'Direct':<20} {effects['direct']:<12.4f} (1/N) × tr(S·β)")
print(f"{'Indirect':<20} {effects['indirect']:<12.4f} (1/N) × Σᵢⱼ S_ij·β (i≠j)")
print(f"{'Total':<20} {effects['total']:<12.4f} (1/N) × Σᵢⱼ S_ij·β")
print("=" * 70)

print("\n📊 ECONOMIC INTERPRETATION")
print("=" * 70)
print(f"\nA 1-unit increase in x_k in one region:")
print(f"\n  ✓ Direct effect:   {effects['direct']:.4f}")
print(f"      → Increases own outcome by {effects['direct']:.3f} (including feedback)")
print(f"\n  ✓ Indirect effect: {effects['indirect']:.4f}")
print(f"      → Increases neighbors' outcomes by {effects['indirect']:.3f} (total spillover)")
print(f"\n  ✓ Total effect:    {effects['total']:.4f}")
print(f"      → System-wide impact is {effects['total']:.3f}")

# Spillover ratio
spillover_pct = 100 * effects['indirect'] / effects['total']
multiplier = effects['total'] / beta_k

print(f"\n🔍 KEY INSIGHTS:")
print(f"  → Spillover share: {spillover_pct:.1f}% of total impact")
print(f"  → Spatial multiplier: {multiplier:.3f}x")
print(f"  → Direct effect is {effects['direct']/beta_k:.2f}× larger than β (due to feedback)")
print(f"  → Ignoring spillovers misses {spillover_pct:.0f}% of policy impact!")
print("=" * 70)

---

## Section 4: Computing Effects with PanelBox (40 minutes)

### Real Data Application

Now let's apply these concepts to a realistic dataset: **regional investment and GDP growth**.

**Research question**: What is the impact of infrastructure investment on regional GDP growth, accounting for spatial spillovers?

**Model**: SAR with fixed effects
$$
\text{growth}_{it} = \rho W \text{growth}_{it} + \beta_1 \text{investment}_{it} + \beta_2 \text{education}_{it} + \beta_3 \text{infrastructure}_{it} + \alpha_i + \varepsilon_{it}
$$

---

In [ ]:
# Generate synthetic regional data for demonstration
np.random.seed(42)

# Create synthetic dataset
n_regions = 20
n_years = 10
n_obs = n_regions * n_years

# Generate grid coordinates for regions
grid_size = int(np.sqrt(n_regions))
coords = [(i, j) for i in range(grid_size) for j in range(grid_size)][:n_regions]

# Build spatial weights matrix (Queen contiguity)
W_data = np.zeros((n_regions, n_regions))
for i, (x1, y1) in enumerate(coords):
    neighbors = []
    for j, (x2, y2) in enumerate(coords):
        if i != j and abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1:
            neighbors.append(j)
    if neighbors:
        W_data[i, neighbors] = 1.0 / len(neighbors)  # Row-normalize

# Generate panel data
data_list = []
for t in range(n_years):
    for i in range(n_regions):
        data_list.append({
            'region_id': i,
            'year': 2010 + t,
            'investment': np.random.gamma(2, 2) + i * 0.1,
            'education': np.random.normal(10, 2) + i * 0.05,
            'infrastructure': np.random.gamma(3, 1.5) + i * 0.08,
        })

df = pd.DataFrame(data_list)

# Generate outcome with spatial lag
rho_true = 0.5
beta_true = np.array([0.8, 1.2, 0.6])  # [investment, education, infrastructure]

y_list = []
for t in range(n_years):
    X_t = df[df['year'] == 2010 + t][['investment', 'education', 'infrastructure']].values
    epsilon = np.random.normal(0, 0.5, n_regions)
    alpha_i = np.arange(n_regions) * 0.1  # Fixed effects
    
    # y = (I - ρW)⁻¹ (Xβ + α + ε)
    I = np.eye(n_regions)
    S = np.linalg.inv(I - rho_true * W_data)
    y_t = S @ (X_t @ beta_true + alpha_i + epsilon)
    y_list.extend(y_t)

df['gdp_growth'] = y_list
df['entity_id'] = df['region_id']
df['time'] = df['year']

print("✓ Synthetic regional investment dataset created")
print(f"  - {n_regions} regions × {n_years} years = {n_obs} observations")
print(f"  - True ρ = {rho_true}")
print(f"  - True β = {beta_true}")
print("\nFirst few observations:")
print(df.head(10))

In [ ]:
# Import PanelBox spatial models
from panelbox.models.spatial import SpatialLag

# Convert W to sparse matrix format expected by PanelBox
from scipy.sparse import csr_matrix
W_sparse = csr_matrix(W_data)

# Estimate SAR model
print("=" * 70)
print("ESTIMATING SAR MODEL")
print("=" * 70)

sar_model = SpatialLag(
    formula="gdp_growth ~ investment + education + infrastructure",
    data=df,
    entity_col='entity_id',
    time_col='time',
    W=W_sparse
)

try:
    sar_results = sar_model.fit(effects='fixed', method='ml')
    print("\n✓ Model estimation successful")
    print(sar_results.summary())
except Exception as e:
    print(f"\n⚠ Model estimation failed: {e}")
    print("\nNote: If PanelBox spatial effects API is not yet implemented,")
    print("we'll demonstrate the methodology with manual calculations.")
    
    # Manual estimation fallback
    sar_results = None

In [ ]:
# Compute spatial effects
# Note: This uses hypothetical API - adjust based on actual PanelBox implementation

print("=" * 70)
print("COMPUTING SPATIAL MARGINAL EFFECTS")
print("=" * 70)

# Manual computation using estimated parameters
# (Replace with PanelBox API when available)

if sar_results is not None:
    # Extract parameters
    rho_hat = sar_results.rho if hasattr(sar_results, 'rho') else 0.5
    params = sar_results.params
else:
    # Use true parameters for demonstration
    rho_hat = rho_true
    params = pd.Series({
        'investment': beta_true[0],
        'education': beta_true[1],
        'infrastructure': beta_true[2]
    })

print(f"\nEstimated ρ = {rho_hat:.4f}")
print(f"\nEstimated coefficients:")
print(params)

# Compute spatial multiplier
I = np.eye(n_regions)
S_hat = np.linalg.inv(I - rho_hat * W_data)

# Compute effects for each variable
effects_table = []

for var in ['investment', 'education', 'infrastructure']:
    beta_k = params[var]
    effects_k = compute_effects_manual(S_hat, beta_k)
    
    effects_table.append({
        'Variable': var,
        'Coefficient (β)': beta_k,
        'Direct': effects_k['direct'],
        'Indirect': effects_k['indirect'],
        'Total': effects_k['total']
    })

effects_df = pd.DataFrame(effects_table)

print("\n" + "=" * 70)
print("SPATIAL MARGINAL EFFECTS")
print("=" * 70)
print(effects_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
print("=" * 70)

In [ ]:
# Detailed interpretation for investment
invest_effects = effects_df[effects_df['Variable'] == 'investment'].iloc[0]

print("=" * 70)
print("INTERPRETATION: INVESTMENT EFFECTS")
print("=" * 70)

print(f"\n📈 Variable: Investment")
print(f"\n  Coefficient (β): {invest_effects['Coefficient (β)']:.4f}")
print(f"    → Initial direct effect (ignoring spatial feedbacks)")

print(f"\n  Direct Effect: {invest_effects['Direct']:.4f}")
print(f"    → Average impact on own region (including feedback)")
print(f"    → {invest_effects['Direct'] / invest_effects['Coefficient (β)']:.2f}× larger than β")

print(f"\n  Indirect Effect: {invest_effects['Indirect']:.4f}")
print(f"    → Average spillover to all other regions")
print(f"    → {100 * invest_effects['Indirect'] / invest_effects['Total']:.1f}% of total impact")

print(f"\n  Total Effect: {invest_effects['Total']:.4f}")
print(f"    → System-wide impact per unit increase in investment")
print(f"    → Spatial multiplier: {invest_effects['Total'] / invest_effects['Coefficient (β)']:.2f}×")

print("\n" + "=" * 70)
print("ECONOMIC MEANING")
print("=" * 70)

print(f"\nA 1% increase in one region's investment:")
print(f"\n  ✓ Boosts own GDP growth by {invest_effects['Direct']:.3f} percentage points")
print(f"  ✓ Boosts neighbors' GDP growth by {invest_effects['Indirect']:.3f} p.p. (total spillover)")
print(f"  ✓ System-wide growth impact: {invest_effects['Total']:.3f} p.p.")

print("\n🔍 Policy Implications:")
spillover_pct = 100 * invest_effects['Indirect'] / invest_effects['Total']
print(f"\n  → {spillover_pct:.1f}% of total impact comes from spillovers")
print(f"  → Regional investment subsidies have large multiplier effects")
print(f"  → Cost-benefit analysis ignoring spillovers underestimates benefits by {spillover_pct:.0f}%")
print(f"  → Coordinated regional policies amplify total impact")

print("=" * 70)

---

## Section 5: SDM Effects (More Complex) (25 minutes)

### Spatial Durbin Model

The **Spatial Durbin Model (SDM)** includes both $Wy$ and $WX$:
$$
y = \rho W y + X\beta + WX\theta + \varepsilon
$$

**Marginal effects**:
$$
\frac{\partial y}{\partial x_k} = (I - \rho W)^{-1} [\beta_k I + \theta_k W]
$$

**Key difference from SAR**:
- SAR: $S(\rho) \beta_k$ — only β enters
- SDM: $S(\rho)[\beta_k I + \theta_k W]$ — both β and θ enter

**Interpretation**:
- $\beta_k$: Direct effect of own x_k (before feedback)
- $\theta_k$: Exogenous spillover from neighbors' x_k
- $\rho$: Endogenous spillover through Wy

---

In [ ]:
# Estimate SDM model
from panelbox.models.spatial import SpatialDurbin

print("=" * 70)
print("ESTIMATING SPATIAL DURBIN MODEL (SDM)")
print("=" * 70)

try:
    sdm_model = SpatialDurbin(
        formula="gdp_growth ~ investment + education + infrastructure",
        data=df,
        entity_col='entity_id',
        time_col='time',
        W=W_sparse
    )
    
    sdm_results = sdm_model.fit(effects='fixed', method='ml')
    print("\n✓ SDM estimation successful")
    print(sdm_results.summary())
except Exception as e:
    print(f"\n⚠ SDM estimation failed: {e}")
    print("\nUsing manual demonstration...")
    sdm_results = None

In [ ]:
# Compute SDM effects manually
def compute_sdm_effects(S, beta_k, theta_k):
    """
    Compute SDM spatial effects.
    
    Parameters
    ----------
    S : ndarray
        Spatial multiplier (I - ρW)⁻¹
    beta_k : float
        Direct coefficient
    theta_k : float
        Spatial lag coefficient
    
    Returns
    -------
    dict
    """
    N = S.shape[0]
    I = np.eye(N)
    
    # Impact matrix: S(ρ) [β I + θ W]
    impact_matrix = S @ (beta_k * I + theta_k * W_data)
    
    direct = np.trace(impact_matrix) / N
    total = impact_matrix.sum() / N
    indirect = total - direct
    
    return {
        'direct': direct,
        'indirect': indirect,
        'total': total,
        'impact_matrix': impact_matrix
    }


# Hypothetical SDM parameters for demonstration
rho_sdm = 0.4
beta_sdm = {'investment': 0.7, 'education': 1.0, 'infrastructure': 0.5}
theta_sdm = {'investment': 0.3, 'education': 0.4, 'infrastructure': 0.2}

S_sdm = np.linalg.inv(np.eye(n_regions) - rho_sdm * W_data)

# Compute effects
effects_sdm_table = []

for var in ['investment', 'education', 'infrastructure']:
    effects_k = compute_sdm_effects(S_sdm, beta_sdm[var], theta_sdm[var])
    
    effects_sdm_table.append({
        'Variable': var,
        'β (direct coef)': beta_sdm[var],
        'θ (lag coef)': theta_sdm[var],
        'Direct Effect': effects_k['direct'],
        'Indirect Effect': effects_k['indirect'],
        'Total Effect': effects_k['total']
    })

effects_sdm_df = pd.DataFrame(effects_sdm_table)

print("=" * 70)
print("SDM SPATIAL EFFECTS")
print("=" * 70)
print(effects_sdm_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
print("=" * 70)

print("\n🔍 Key Observation:")
print("  → Direct effect ≠ β (includes feedback and θW component)")
print("  → Indirect effect includes both endogenous (ρ) and exogenous (θ) spillovers")
print("  → SDM effects typically differ substantially from SAR")

In [ ]:
# Compare SAR vs SDM effects
comparison = pd.DataFrame({
    'Variable': ['investment', 'education', 'infrastructure'],
    'SAR Direct': effects_df['Direct'].values,
    'SDM Direct': effects_sdm_df['Direct Effect'].values,
    'SAR Indirect': effects_df['Indirect'].values,
    'SDM Indirect': effects_sdm_df['Indirect Effect'].values,
    'SAR Total': effects_df['Total'].values,
    'SDM Total': effects_sdm_df['Total Effect'].values,
})

print("=" * 70)
print("SAR vs SDM EFFECTS COMPARISON")
print("=" * 70)
print(comparison.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
print("=" * 70)

print("\n📊 Observations:")
print("\n  1. Direct effects differ due to WXθ terms in SDM")
print("  2. Indirect effects typically larger in SDM (exogenous spillovers θ)")
print("  3. Model choice affects effect estimates — use specification tests!")
print("  4. SDM is more flexible (nests SAR and SLX)")

---

## Section 6: Inference - Simulation vs Delta Method (20 minutes)

### Two Approaches to Standard Errors

Computing standard errors for spatial effects is non-trivial because effects are **nonlinear functions** of parameters (ρ, β, θ).

#### 1. Simulation-Based Inference (Recommended)

**Algorithm**:
1. Draw parameters from asymptotic distribution: $(\hat{\rho}, \hat{\beta}) \sim N(\theta, \hat{V})$
2. For each draw, compute effects: direct, indirect, total
3. Empirical distribution → percentile confidence intervals

**Pros**: Robust to nonlinearity, exact for large samples  
**Cons**: Computationally intensive

#### 2. Delta Method

**Approach**: First-order Taylor approximation
$$
\text{Var}(g(\hat{\theta})) \approx \nabla g(\hat{\theta})' \text{Var}(\hat{\theta}) \nabla g(\hat{\theta})
$$

**Pros**: Fast, analytical  
**Cons**: Less accurate for high |ρ|, complex derivatives

---

### Recommendation

- Use **simulation** for final results (more accurate)
- Use **delta method** for quick checks or very large datasets

---

In [ ]:
# Simulation-based inference
def simulation_inference(rho, beta_k, W, vcov, n_simulations=1000):
    """
    Compute effects confidence intervals via simulation.
    
    Parameters
    ----------
    rho : float
        Estimated spatial parameter
    beta_k : float
        Estimated coefficient
    W : ndarray
        Spatial weights matrix
    vcov : ndarray (2×2)
        Variance-covariance matrix [rho, beta_k]
    n_simulations : int
        Number of simulation draws
    
    Returns
    -------
    dict
        Mean and 95% CI for direct, indirect, total
    """
    N = W.shape[0]
    I = np.eye(N)
    
    # Draw from asymptotic distribution
    params = np.random.multivariate_normal([rho, beta_k], vcov, n_simulations)
    
    direct_draws = []
    indirect_draws = []
    total_draws = []
    
    for rho_draw, beta_draw in params:
        # Ensure stability: |ρ| < 1
        if abs(rho_draw) >= 1:
            continue
        
        S = np.linalg.inv(I - rho_draw * W)
        effects = compute_effects_manual(S, beta_draw)
        
        direct_draws.append(effects['direct'])
        indirect_draws.append(effects['indirect'])
        total_draws.append(effects['total'])
    
    # Compute percentiles
    return {
        'direct': {
            'mean': np.mean(direct_draws),
            'ci_lower': np.percentile(direct_draws, 2.5),
            'ci_upper': np.percentile(direct_draws, 97.5),
            'std': np.std(direct_draws)
        },
        'indirect': {
            'mean': np.mean(indirect_draws),
            'ci_lower': np.percentile(indirect_draws, 2.5),
            'ci_upper': np.percentile(indirect_draws, 97.5),
            'std': np.std(indirect_draws)
        },
        'total': {
            'mean': np.mean(total_draws),
            'ci_lower': np.percentile(total_draws, 2.5),
            'ci_upper': np.percentile(total_draws, 97.5),
            'std': np.std(total_draws)
        }
    }


# Hypothetical variance-covariance matrix
vcov_invest = np.array([
    [0.01, 0.002],   # Var(ρ), Cov(ρ, β)
    [0.002, 0.04]    # Cov(ρ, β), Var(β)
])

# Run simulation
print("Running simulation-based inference (1000 draws)...")
sim_results = simulation_inference(
    rho=rho_hat,
    beta_k=params['investment'],
    W=W_data,
    vcov=vcov_invest,
    n_simulations=1000
)

print("\n" + "=" * 70)
print("SIMULATION-BASED INFERENCE RESULTS (Investment)")
print("=" * 70)
print(f"\n{'Effect':<15} {'Mean':<12} {'Std Error':<12} {'95% CI':<25}")
print("-" * 70)

for effect_type in ['direct', 'indirect', 'total']:
    res = sim_results[effect_type]
    ci_str = f"[{res['ci_lower']:.4f}, {res['ci_upper']:.4f}]"
    print(f"{effect_type.capitalize():<15} {res['mean']:<12.4f} {res['std']:<12.4f} {ci_str:<25}")

print("=" * 70)
print("\n✓ Simulation completed")
print("  → 95% confidence intervals account for parameter uncertainty")
print("  → More accurate than delta method for nonlinear effects")

In [ ]:
# Delta method (simplified demonstration)
# Note: Full delta method requires analytical derivatives of S(ρ)

print("=" * 70)
print("DELTA METHOD vs SIMULATION COMPARISON")
print("=" * 70)

print("\n🔬 Simulation Method:")
print("  ✓ More accurate (exact in large samples)")
print("  ✓ No approximation error")
print("  ✓ Robust to high ρ")
print("  ✗ Computationally intensive")

print("\n📐 Delta Method:")
print("  ✓ Fast (analytical)")
print("  ✓ Works for large datasets")
print("  ✗ First-order approximation (less accurate)")
print("  ✗ Requires complex derivatives")
print("  ✗ Less accurate when |ρ| > 0.5")

print("\n🎯 Recommendation:")
print(f"  → For ρ = {rho_hat:.2f}: Use SIMULATION (recommended)")
print("  → Delta method acceptable only if |ρ| < 0.3 and speed is critical")

print("=" * 70)

---

## Section 7: Visualizing Effects (20 minutes)

Effective visualization is crucial for communicating spatial effects to policymakers and stakeholders.

### Key Visualizations

1. **Effects decomposition bar chart**: Compare direct, indirect, total across variables
2. **Spillover decay plot**: Show how effects decay with spatial order
3. **Confidence intervals**: Display uncertainty in effects estimates

---

In [ ]:
# Create output directory
output_dir = Path("../outputs/figures")
output_dir.mkdir(parents=True, exist_ok=True)

# Effects decomposition plot
fig, ax = plt.subplots(figsize=(12, 6))

variables = effects_df['Variable'].values
direct = effects_df['Direct'].values
indirect = effects_df['Indirect'].values
total = effects_df['Total'].values

x = np.arange(len(variables))
width = 0.25

bars1 = ax.bar(x - width, direct, width, label='Direct', 
               alpha=0.8, edgecolor='black', color='steelblue')
bars2 = ax.bar(x, indirect, width, label='Indirect', 
               alpha=0.8, edgecolor='black', color='coral')
bars3 = ax.bar(x + width, total, width, label='Total', 
               alpha=0.8, edgecolor='black', color='seagreen')

ax.set_xlabel('Variable', fontsize=13, fontweight='bold')
ax.set_ylabel('Marginal Effect', fontsize=13, fontweight='bold')
ax.set_title('Spatial Marginal Effects Decomposition (SAR Model)', 
             fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels([v.capitalize() for v in variables], fontsize=11)
ax.axhline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.legend(fontsize=12, loc='upper left', framealpha=0.9)
ax.grid(True, axis='y', alpha=0.3, linestyle=':')

# Add value labels on bars
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(output_dir / 'nb06_effects_decomposition.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Effects decomposition plot saved")

In [ ]:
# Spillover decay plot
# Shows how spillover effects decay with spatial order (k-th order neighbors)

fig, ax = plt.subplots(figsize=(10, 6))

orders = np.arange(0, 10)
beta_invest = params['investment']

# Spillover at order k = ρ^k × β
# (This is a simplification; actual decay depends on W^k structure)
spillover_by_order = [beta_invest * (rho_hat ** k) for k in orders]

ax.plot(orders, spillover_by_order, marker='o', linewidth=2.5, 
        markersize=8, color='steelblue', label='Investment spillover')
ax.fill_between(orders, 0, spillover_by_order, alpha=0.2, color='steelblue')

ax.set_xlabel('Spatial Order (k-th neighbor)', fontsize=13, fontweight='bold')
ax.set_ylabel('Spillover Effect', fontsize=13, fontweight='bold')
ax.set_title(f'Spatial Spillover Decay (ρ = {rho_hat:.3f})', 
             fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle=':')
ax.axhline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.set_xticks(orders)

# Add annotations
ax.annotate(f'Direct effect\n(order 0): {spillover_by_order[0]:.3f}',
            xy=(0, spillover_by_order[0]), xytext=(2, spillover_by_order[0] * 1.2),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, ha='center')

ax.annotate(f'Spillover rapidly\ndecays to ~0',
            xy=(5, spillover_by_order[5]), xytext=(7, spillover_by_order[0] * 0.3),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, ha='center')

plt.tight_layout()
plt.savefig(output_dir / 'nb06_spillover_decay.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Spillover decay plot saved")
print("\n📊 Interpretation:")
print(f"  → Spillovers decay exponentially at rate ρ^k")
print(f"  → With ρ = {rho_hat:.2f}, effects negligible beyond 5th-order neighbors")
print(f"  → Higher ρ → slower decay, more global spillovers")

In [ ]:
# Effects with confidence intervals
fig, ax = plt.subplots(figsize=(10, 7))

effect_types = ['Direct', 'Indirect', 'Total']
effect_keys = ['direct', 'indirect', 'total']

means = [sim_results[key]['mean'] for key in effect_keys]
ci_lower = [sim_results[key]['ci_lower'] for key in effect_keys]
ci_upper = [sim_results[key]['ci_upper'] for key in effect_keys]

y_pos = np.arange(len(effect_types))

ax.barh(y_pos, means, xerr=[np.array(means) - np.array(ci_lower),
                             np.array(ci_upper) - np.array(means)],
        alpha=0.7, edgecolor='black', linewidth=1.5,
        color=['steelblue', 'coral', 'seagreen'],
        capsize=5, error_kw={'linewidth': 2})

ax.set_yticks(y_pos)
ax.set_yticklabels(effect_types, fontsize=12)
ax.set_xlabel('Effect Estimate (95% CI)', fontsize=13, fontweight='bold')
ax.set_title('Investment Effects with Confidence Intervals\n(Simulation-Based Inference)',
             fontsize=15, fontweight='bold', pad=20)
ax.axvline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax.grid(True, axis='x', alpha=0.3, linestyle=':')

# Add value labels
for i, (mean, lower, upper) in enumerate(zip(means, ci_lower, ci_upper)):
    ax.text(mean, i, f'  {mean:.3f}', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'nb06_effects_ci.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Confidence interval plot saved")

---

## Section 8: Policy Application - Cost-Benefit Analysis (25 minutes)

### Real-World Policy Evaluation

**Scenario**: Government considers $10 million infrastructure investment in one region.

**Question**: What is the total economic impact, including spillovers?

**Analysis**:
1. Compute direct, indirect, total effects
2. Translate effects to dollar values
3. Calculate spatial multiplier
4. Compare benefit vs cost
5. Evaluate spillover share

---

In [ ]:
# Policy simulation: Infrastructure investment
print("=" * 70)
print("POLICY SIMULATION: INFRASTRUCTURE INVESTMENT")
print("=" * 70)

# Parameters
investment_amount = 10_000_000  # $10M investment
avg_regional_gdp = df.groupby('region_id')['gdp_growth'].mean().mean() * 1_000_000_000  # Hypothetical $1B average GDP

# Extract infrastructure effects
infra_effects = effects_df[effects_df['Variable'] == 'infrastructure'].iloc[0]

direct_effect = infra_effects['Direct']
indirect_effect = infra_effects['Indirect']
total_effect = infra_effects['Total']

print(f"\n💰 Investment: ${investment_amount:,.0f} in infrastructure")
print(f"📊 Average regional GDP: ${avg_regional_gdp:,.0f}")

print("\n" + "=" * 70)
print("MARGINAL EFFECTS (from SAR estimation)")
print("=" * 70)
print(f"\n  Direct effect:   {direct_effect:.4f}")
print(f"  Indirect effect: {indirect_effect:.4f}")
print(f"  Total effect:    {total_effect:.4f}")

# Convert to dollar impacts
# Assuming effects are in percentage point changes in GDP growth
# Simplified: impact = effect × avg_GDP × scale_factor
scale_factor = investment_amount / avg_regional_gdp  # Investment as fraction of GDP

direct_value = direct_effect * avg_regional_gdp * scale_factor
indirect_value = indirect_effect * avg_regional_gdp * scale_factor
total_value = total_effect * avg_regional_gdp * scale_factor

print("\n" + "=" * 70)
print("ESTIMATED GDP IMPACT")
print("=" * 70)
print(f"\n  Direct (own region):     ${direct_value:>15,.0f}")
print(f"  Indirect (neighbors):    ${indirect_value:>15,.0f}")
print(f"  Total (system-wide):     ${total_value:>15,.0f}")

# Multiplier
multiplier = total_value / investment_amount
spillover_share = 100 * indirect_value / total_value

print("\n" + "=" * 70)
print("KEY METRICS")
print("=" * 70)
print(f"\n  🔢 Spatial Multiplier: {multiplier:.2f}×")
print(f"     → Every $1 invested generates ${multiplier:.2f} in total GDP impact")

print(f"\n  🌐 Spillover Share: {spillover_share:.1f}%")
print(f"     → {spillover_share:.0f}% of total benefits accrue to neighboring regions")

print(f"\n  💡 Own Region Share: {100 - spillover_share:.1f}%")
print(f"     → Only {100 - spillover_share:.0f}% of benefits stay in investing region")

print("\n" + "=" * 70)
print("POLICY IMPLICATIONS")
print("=" * 70)

print(f"\n  ✓ Investment is economically justified (multiplier > 1)")
print(f"\n  ✓ Ignoring spillovers underestimates total benefits by {spillover_share:.0f}%")
print(f"     → Non-spatial analysis would miss ${indirect_value:,.0f} in spillover benefits!")

print(f"\n  ✓ Large spillovers suggest:")
print(f"     • Regional coordination enhances efficiency")
print(f"     • Cost-sharing with neighbors may be justified")
print(f"     • Investing region captures only {100 - spillover_share:.0f}% of benefits")

print(f"\n  ✓ Free-rider problem:")
print(f"     • Neighbors benefit without paying")
print(f"     • May lead to under-investment without coordination")
print(f"     • Federal/state co-financing can internalize spillovers")

print("\n" + "=" * 70)

In [ ]:
# Visualize policy impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Benefit breakdown
benefits = [direct_value, indirect_value]
labels = ['Direct\n(Own Region)', 'Indirect\n(Spillovers)']
colors = ['steelblue', 'coral']

wedges, texts, autotexts = ax1.pie(benefits, labels=labels, autopct='%1.1f%%',
                                     colors=colors, startangle=90,
                                     textprops={'fontsize': 11, 'fontweight': 'bold'})
ax1.set_title('Distribution of Total Benefits\n($10M Infrastructure Investment)',
              fontsize=13, fontweight='bold', pad=20)

# Panel 2: Multiplier comparison
scenarios = ['Without\nSpillovers', 'With\nSpillovers\n(Actual)']
multipliers = [direct_value / investment_amount, multiplier]

bars = ax2.bar(scenarios, multipliers, color=['lightgray', 'seagreen'],
               alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Economic Multiplier', fontsize=12, fontweight='bold')
ax2.set_title('Spatial Multiplier Effect', fontsize=13, fontweight='bold', pad=20)
ax2.axhline(1, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='Break-even')
ax2.grid(True, axis='y', alpha=0.3, linestyle=':')
ax2.legend(fontsize=10)

# Add value labels
for bar, mult in zip(bars, multipliers):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{mult:.2f}×',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'nb06_policy_impact.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Policy impact visualization saved")

---

## Section 9: Common Pitfalls (15 minutes)

### What NOT to Do ❌

---

In [ ]:
print("=" * 70)
print("COMMON MISTAKES IN SPATIAL MARGINAL EFFECTS")
print("=" * 70)

print("\n❌ MISTAKE #1: Interpreting β as marginal effect")
print("   Wrong:  'Investment coefficient is 0.8, so marginal effect is 0.8'")
print("   Right:  'β = 0.8 is PART of effect; marginal effect = S(ρ)β'")
print(f"   Example: β = {params['investment']:.3f}, but direct effect = {invest_effects['Direct']:.3f}")
print(f"            (Direct effect is {invest_effects['Direct'] / params['investment']:.2f}× larger!)")

print("\n❌ MISTAKE #2: Ignoring feedback loops")
print("   Wrong:  'Direct effect = β'")
print("   Right:  'Direct effect = average diagonal of S(ρ)β (includes feedback)'")
print("   Key:    Feedback through ρW amplifies direct effect")

print("\n❌ MISTAKE #3: Reporting only total effect")
print("   Wrong:  'The effect is 1.5'")
print("   Right:  'Direct = 0.9, Indirect = 0.6, Total = 1.5'")
print("   Why:    Policy needs to know own vs spillover effects")

print("\n❌ MISTAKE #4: Using delta method for high ρ")
print("   Wrong:  'Delta method is always fine'")
print("   Right:  'Use simulation when |ρ| > 0.5'")
print(f"   Example: ρ = {rho_hat:.2f} → simulation preferred")

print("\n❌ MISTAKE #5: Not computing effects at all!")
print("   Wrong:  Stop after seeing SAR coefficient table")
print("   Right:  ALWAYS compute and report direct/indirect/total effects")
print("   Why:    Coefficients are uninterpretable in spatial models")

print("\n❌ MISTAKE #6: Comparing SAR and OLS coefficients")
print("   Wrong:  'SAR β is smaller than OLS β, so effect is smaller'")
print("   Right:  'Compare SAR total effect with OLS coefficient'")
print("   Why:    SAR total effect often larger due to multiplier")

print("\n❌ MISTAKE #7: Treating all effects as 'spillovers'")
print("   Wrong:  'Spatial models capture spillovers = total effect'")
print("   Right:  'Only INDIRECT effect is spillover; direct ≠ spillover'")
print("   Why:    Terminology matters for policy communication")

print("\n" + "=" * 70)
print("BEST PRACTICES ✓")
print("=" * 70)

print("\n  1. ✓ ALWAYS compute effects after spatial model estimation")
print("  2. ✓ Report direct, indirect, AND total effects")
print("  3. ✓ Use simulation-based inference (1000+ draws)")
print("  4. ✓ Include confidence intervals")
print("  5. ✓ Visualize effects decomposition")
print("  6. ✓ Interpret spillover share for policy")
print("  7. ✓ Compare effects across model specifications")
print("  8. ✓ Never interpret β coefficients directly")

print("\n" + "=" * 70)

---

## Section 10: Summary and Next Steps (5 minutes)

### Key Takeaways 🎯

#### 1. Fundamental Insight
**Coefficients β ≠ marginal effects** in spatial models due to:
- Feedback loops (y_i → y_j → y_i)
- Spatial multiplier S(ρ) = (I - ρW)⁻¹
- Endogenous interactions

#### 2. Three Types of Effects
- **Direct**: Average own effect (includes feedback)
- **Indirect**: Average spillover to neighbors
- **Total**: System-wide impact (direct + indirect)

#### 3. Computation
- SAR: $\partial y / \partial x_k = S(\rho) \beta_k$
- SDM: $\partial y / \partial x_k = S(\rho)[\beta_k I + \theta_k W]$
- Use simulation-based inference for accuracy

#### 4. Policy Relevance
- Spillovers can be 30-70% of total impact
- Spatial multiplier amplifies policy effects
- Ignoring spillovers severely underestimates benefits

#### 5. Reporting
**ALWAYS report**:
- Direct, indirect, total effects
- Confidence intervals (simulation-based)
- Spillover share (%)
- Spatial multiplier

---

### Checklist for Applied Work ✅

Before publishing spatial model results, ensure:

- [ ] Computed direct, indirect, total effects (not just β)
- [ ] Used simulation-based inference (≥1000 draws)
- [ ] Reported confidence intervals for all effects
- [ ] Visualized effects decomposition
- [ ] Interpreted spillover share for policy
- [ ] Compared effects across model specifications (SAR, SDM, etc.)
- [ ] Explained spatial multiplier to non-technical audience
- [ ] Did NOT interpret β coefficients as marginal effects!

---

### What's Next?

#### Notebook 07: Dynamic Spatial Panels
- Combine spatial and temporal dynamics
- Dynamic spatial panel models
- Long-run vs short-run spatial effects

#### Notebook 08: Specification Tests
- Choosing between SAR, SEM, SDM, SDEM
- LM tests, LR tests, Wald tests
- Model selection strategies

---

### Further Reading 📚

1. **LeSage, J. P., & Pace, R. K. (2009)**. *Introduction to Spatial Econometrics*. CRC Press.
   - Chapter 2: Interpretation of spatial models
   - Section 2.8: Direct and indirect effects

2. **Elhorst, J. P. (2014)**. *Spatial Econometrics: From Cross-Sectional Data to Spatial Panels*. Springer.
   - Chapter 2: Spatial effects decomposition

3. **Halleck Vega, S., & Elhorst, J. P. (2015)**. "The SLX model." *Journal of Regional Science*, 55(3), 339-363.
   - Comparison of effects across spatial models

---

In [ ]:
# Final summary
print("=" * 70)
print("CONGRATULATIONS! 🎉")
print("=" * 70)

print("\nYou have completed the Spatial Marginal Effects tutorial!\n")

print("You now know how to:")
print("  ✓ Compute direct, indirect, and total spatial effects")
print("  ✓ Interpret the spatial multiplier S(ρ)")
print("  ✓ Distinguish SAR from SDM effects")
print("  ✓ Use simulation-based inference")
print("  ✓ Evaluate policy impacts with spillovers")
print("  ✓ Avoid common pitfalls in spatial analysis\n")

print("Remember:")
print("  🚨 NEVER interpret β coefficients directly!")
print("  🚨 ALWAYS compute and report direct/indirect/total effects!")
print("  🚨 Spillovers matter — don't ignore them!\n")

print("Next steps:")
print("  → Notebook 07: Dynamic Spatial Panels")
print("  → Notebook 08: Specification Tests\n")

print("=" * 70)
print("Happy modeling! 🚀")
print("=" * 70)